In [7]:
import pickle
import pandas as pd

In [8]:
loaded_model = pickle.load(open('estructuraDatos.sav', 'rb'))
idexFiles = loaded_model['idexFiles']
vectorizer = loaded_model['vectorizer']
matrix = loaded_model['matriz']

In [14]:
# Encontrar los documentos que contiene una palabra en particular
def encontrarDoc(palabra):
    col = vectorizer.vocabulary_[palabra]
    matx = matrix[:,col]
    indx = matx.nonzero()[0]
    lista =indx.tolist() 
    dfresult = pd.DataFrame()
    for i in range(len(lista)):
        auxres= pd.DataFrame({'NombreArchivo': idexFiles[lista[i]], 'Frecuencia': [matx.data[i]]})
        dfresult = pd.concat([dfresult, auxres])
    dfresult.sort_values('Frecuencia',ascending = False,inplace = True)
    return dfresult

In [15]:
encontrarDoc('accident')

,NombreArchivo,Frecuencia
0,1411.0440.txt,6
0,1309.2348.txt,3
0,1306.0760.txt,2
0,1504.02141.txt,1
0,1502.01410.txt,1
0,1503.00491.txt,1
0,1203.5188.txt,1
0,1501.01042.txt,1
0,1307.7430.txt,1
0,1409.2248.txt,1
